# Sync Youtube Music to Tidal
## API Authentication
Authenticate with both YTMusic API and Tidal API. Note if the Tidal key is out of date the console will spit out a link you need to follow. Once authenticated it will provide a new session ID, enter this in the code below.

In [1]:
from ytmusicapi import YTMusic
import tidalapi
import csv
from dotenv import load_dotenv

# Authenticate Youtube
yt = YTMusic('headers_auth.json')

# Load Tidal Authentication Information
load_dotenv()

TIDAL_TOKEN_TYPE = os.getenv('TIDAL_TOKEN_TYPE')
TIDAL_SESSION_ID = os.getenv('TIDAL_SESSION_ID')
TIDAL_ACCESS_TOKEN = os.getenv('TIDAL_ACCESS_TOKEN')

# Authenticate Tidal
tidal = tidalapi.Session()
if not tidal.load_oauth_session(TIDAL_SESSION_ID, TIDAL_TOKEN_TYPE, TIDAL_ACCESS_TOKEN):
    tidal.login_oauth_simple()

    print('Authenticated! Please add the following to your .env')
    print('Session ID: ' + tidal.session_id)
    print('Token Type: ' + tidal.token_type)
    print('Access Token: ' + tidal.access_token)
else:
    print('Authenticated!')

## Getting Youtube Music Tracks
Get the youtube music tracks and parse them as a list of dictionaries containing the song name, artist name and album name. Then, run through this dictionary, searching each song in Tidal and trying to make a match.

In [4]:
yt_library = []
for item in yt.get_library_songs(10):
    yt_library.append({'name': item['title'], 'artist': item['artists'][0]['name'], 'album': item['album']['name']})

matches = []

# Loop over every item in the library
for item in yt_library:
    best_quality = 0
    best_quality_details = {'input_name': item['name'], 'input_artist': item['artist'], 'input_album': item['album'], 'match_quality': 0}

    # Loop over every track that is returned in the search
    for track in tidal.search('track', item['name'], 100).tracks:
        quality = 0
        
        # Make sure the name matches and skip if it doesn't, add quality depending on match
        if track.name.lower() == item['name'].lower():
            quality += 100
            if track.artist.name.lower() == item['artist'].lower():
                quality += 10
            if track.album.name.lower() == item['album'].lower():
                quality += 1
        else:
            continue

        # If the quality is 111 it is a perfect match so we can add it and skip the rest of the search
        if quality == 111:
            best_quality = quality
            best_quality_details.update({'output_name': track.name, 'output_id': track.id, 'output_artist': track.artist.name, 'output_album': track.album.name, 'match_quality': quality})
            break

        # Otherwise remember the best one
        elif quality > best_quality:
            best_quality = quality
            best_quality_details.update({'output_name': track.name, 'output_id': track.id, 'output_artist': track.artist.name, 'output_album': track.album.name, 'match_quality': quality})

    matches.append(best_quality_details)


## Adding the Matching Searches to Tidal Favourites
Run through the list of matching searches and add those that meet a certain threshold to your Tidal favourites.

In [6]:
# tidal.user.favorites.add_track(132710021)
for match in matches:
    if match['match_quality'] >= 110:
        print('Adding: ' + match['input_name'] + ' by ' + match['input_artist'] + ' to Tidal. Quality: ' + str(match['match_quality']))
        if tidal.user.favorites.add_track(match['output_id']):
            match['status'] = 'Success'
        else:
            match['status'] = 'Failed'
    else:
        match['status'] = 'No Match'


Adding: NDA by Billie Eilish to Tidal. Quality: 111
Adding: Emoji by Au/Ra to Tidal. Quality: 111
Adding: Kill Me With Your Love by One True God to Tidal. Quality: 111
Adding: Temper by Vera Blue to Tidal. Quality: 111
Adding: Basics by Easy Life to Tidal. Quality: 111
Adding: C'est la vie by Weathers to Tidal. Quality: 111
Adding: He Don't Love Me by Winona Oak to Tidal. Quality: 111
Adding: Why Do You Love Me by Charlotte Lawrence to Tidal. Quality: 111
Adding: Feel Me by Selena Gomez to Tidal. Quality: 111
Adding: No Roots by Alice Merton to Tidal. Quality: 111
Adding: Angry Too by Lola Blanc to Tidal. Quality: 111
Adding: Love Like Mine by Stela Cole to Tidal. Quality: 111
Adding: I Feel Like I'm Drowning by Two Feet to Tidal. Quality: 111
Adding: Human by Sevdaliza to Tidal. Quality: 111
Adding: Find an Island by BENEE to Tidal. Quality: 111
Adding: A Little Messed Up by june to Tidal. Quality: 111
Adding: Controlla by Yarimov to Tidal. Quality: 111


## Make a Report of Songs Added
Create a csv report of what songs were added as favourites - and more importantly which were not.

In [9]:
fieldnames = ['status', 'match_quality', 'input_name', 'input_artist', 'input_album', 'output_name', 'output_artist', 'output_album', 'output_id']

with open('sync_report.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(matches)